In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip
import astropy.units as u
from astropy.table import Table
import numpy as np
import os
import pandas as pd
from glob import glob

from split_image import bin_image
from profound_estimate_sky import calculate_sky_profound
from make_diagnostic import make_plots

In [2]:
# Read latest zeropoint corrections
# https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-performance/nircam-absolute-flux-calibration-and-zeropoints
best_zp_df = Table.read('/Users/rosaliaobrien/pearls/sky_measurements/NRC_ZPs_1126pmap.txt', 
           format = 'ascii', data_start = 1, guess = False, delimiter = '|',
           names = ['na1', 'pupil+filter', 'detector', 'PHOTMJSR', 'zp_vega', 'vega_Jy', 'zp_vega-sirius', 'vega-sirius_Jy',
                    'zp_AB', 'mean_pix_sr', 'na2']) 

# Add a column with just the filter names
best_zp_df['filter'] = [name.split('+')[1] for name in best_zp_df['pupil+filter']]

# Define function to get ratio of new zeropoint / old zeropoint
def sky_zp_corr(new_ZP, photmjsr):
    
    new_ZP = new_ZP*u.MJy/u.sr
    
    ratio = new_ZP/photmjsr
    
    return ratio.value

In [2]:
# Read in info about the profound images
### This is in the Pro1oF directory produced from JUMPROPE
cal_info_tab = pd.read_csv('/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_info.csv')
cal_info_tab['root'] = cal_info_tab['file'].str.split('_cal', expand = True)[0]
cal_info_tab

,full,file,stub,path,ext,PHOTMJSR,PIXAR_SR,CRVAL1,CRVAL2,CD1_1,...,ext.1,VISIT_ID,DATE-OBS,DETECTOR,FILTER,EFFEXPTM,CRDS_CTX,MAGZERO,MAGZERO_FIX,root
0,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw01176231001_02101_00001_nrca1_cal_Pro1oF_sky...,jw01176231001_02101_00001_nrca1_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,3.179000,2.243274e-14,177.410871,22.408895,6.490443e-08,...,1,1176231001,2024-01-24,NRCA1,F090W,837.468,jwst_1252.pmap,28.022794,28.022794,jw01176231001_02101_00001_nrca1
1,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw01176231001_02101_00001_nrca2_cal_Pro1oF_sky...,jw01176231001_02101_00001_nrca2_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,3.405000,2.230000e-14,177.390524,22.408686,3.102602e-08,...,1,1176231001,2024-01-24,NRCA2,F090W,837.468,jwst_1252.pmap,28.029238,28.029238,jw01176231001_02101_00001_nrca2
2,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw01176231001_02101_00001_nrca3_cal_Pro1oF_sky...,jw01176231001_02101_00001_nrca3_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,3.132000,2.264701e-14,177.410973,22.389831,-1.536250e-08,...,1,1176231001,2024-01-24,NRCA3,F090W,837.468,jwst_1252.pmap,28.012473,28.012473,jw01176231001_02101_00001_nrca3
3,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw01176231001_02101_00001_nrca4_cal_Pro1oF_sky...,jw01176231001_02101_00001_nrca4_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,3.008000,2.250000e-14,177.390536,22.389969,-1.774885e-08,...,1,1176231001,2024-01-24,NRCA4,F090W,837.468,jwst_1252.pmap,28.019544,28.019544,jw01176231001_02101_00001_nrca4
4,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw01176231001_02101_00001_nrcalong_cal_Pro1oF_...,jw01176231001_02101_00001_nrcalong_cal_Pro1oF_...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,0.402000,9.209440e-14,177.400645,22.399286,2.555108e-08,...,1,1176231001,2024-01-24,NRCALONG,F444W,837.468,jwst_1252.pmap,26.489417,26.489417,jw01176231001_02101_00001_nrcalong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw03362011001_16201_00001_nrcb1_cal_Pro1oF_sky...,jw03362011001_16201_00001_nrcb1_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,45.452999,2.230000e-14,177.433594,22.323447,4.567656e-06,...,1,3362011001,2024-05-09,NRCB1,F187N,450.944,jwst_1252.pmap,28.029238,28.029238,jw03362011001_16201_00001_nrcb1
1864,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw03362011001_16201_00001_nrcb2_cal_Pro1oF_sky...,jw03362011001_16201_00001_nrcb2_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,43.657001,2.290000e-14,177.416384,22.333465,4.673340e-06,...,1,3362011001,2024-05-09,NRCB2,F187N,450.944,jwst_1252.pmap,28.000411,28.000411,jw03362011001_16201_00001_nrcb2
1865,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw03362011001_16201_00001_nrcb3_cal_Pro1oF_sky...,jw03362011001_16201_00001_nrcb3_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,43.972000,2.250000e-14,177.422813,22.307466,4.480450e-06,...,1,3362011001,2024-05-09,NRCB3,F187N,450.944,jwst_1252.pmap,28.019544,28.019544,jw03362011001_16201_00001_nrcb3
1866,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/...,jw03362011001_16201_00001_nrcb4_cal_Pro1oF_sky...,jw03362011001_16201_00001_nrcb4_cal_Pro1oF_sky...,/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm,2,39.776001,2.310000e-14,177.405474,22.317402,4.581699e-06,...,1,3362011001,2024-05-09,NRCB4,F187N,450.944,jwst_1252.pmap,27.990970,27.990970,jw03362011001_16201_00001_nrcb4


In [3]:
# Make function to get certain keywords from the above table
key_list = ['CRVAL1', 'CRVAL2', 'PHOTMJSR', 'DATE-OBS', 'VISIT_ID', 'DETECTOR', 'FILTER', 'EFFEXPTM']

def get_header_info(file_root):
    
    dic = {}
    for key in key_list:

        row = cal_info_tab[cal_info_tab['root'] == file_root]

        dic[key] = [row[key].item()]

    return dic

In [44]:
hdu = fits.open('/Volumes/JWSTSIM2/m1149/Pro1oF/cal_sky_renorm/jw03362011001_16201_00001_nrcb4_cal_Pro1oF_sky_rem.fits')

In [11]:
# Get files you will use for sky measurements
### I think these files need to be the cal_sky_renorm files from the Pro1oF directory after running JUMPROPE
file_list = glob('/Volumes/JWSTSIM2/m1149/Pro1oF/cal/jw02883*.fits')
len(file_list)

df = pd.DataFrame([])

for i, file in enumerate(file_list):

    basefile = os.path.basename(file) # file name
#     fullroot = basefile.split('.')[0] # root name with the _cal
    root = basefile.split('.')[0].split('_cal')[0]

    # Make simple dataframe with file name and root name
    file_df = pd.DataFrame({'file': [basefile], 'root': [root]})
    
    # Get information from headers
#     file_root = os.path.basename(file)[:25]
    phdr_dic = get_header_info(root)
    
    print(i, root)
    
    # Update sci data with updated zeropoints
    flter = phdr_dic['FILTER']
    detector = phdr_dic['DETECTOR']
    
    print(detector, flter)

    # Get data
    hdu = fits.open(file)

    # The profound corrected images have the sky subtracted, 
    # so you need to add the sky back before doing sky measurements
    sky_data = hdu['SCI'].data 
    #rms_data = hdu['SKYRMS'].data
    
    # Grab new zeropoint
#     new_ZP = best_zp_df[ (best_zp_df['filter'] == flter) & 
#                          (best_zp_df['detector'] == detector)]['PHOTMJSR'].value[0]
    
    # Adjust sky and rms maps to match new zeropoint
#     ratio = sky_zp_corr(new_ZP = new_ZP, photmjsr = phdr_dic['PHOTMJSR'][0]*u.MJy/u.sr)
#     sky_data = sky_data_og*ratio
#     rms_data = rms_data_og*ratio

    # Calculate sky level
    sky_dic = calculate_sky_profound(sky_data = sky_data, rms_data = None, ### Have the rms calculated already when running calculate_sky_profound from measureskyregion.py, and don't have hdu['SKYRMS']
                                      bin_size = 64, 
                                     dq_data = hdu[3].data, dq_good_list = [0], has_DQ = True, 
                                     dq_fraction = 0.2, percentile = 50)

    hdu.close()

    # Get keywords from headers and stuff
    phdr_df = pd.DataFrame.from_dict(phdr_dic)
    sky_df = pd.DataFrame.from_dict(sky_dic)

    sky_df_forcsv = sky_df[['calc_sky', 'calc_rms', 'N_total_regions', 'N_bad_regions',
                           'N_bad_px_regions', 'N_good_regions']]

    final_df = pd.concat([file_df, sky_df_forcsv, phdr_df], axis = 1)
    #df = df.append(final_df)
    final_df.to_csv(f'./output/{root}_calcSky_output.csv', index = False)

    make_plots(sky_data, sky_df['cutouts'].to_list()[0], sky_df['lowest5perc_ind'].to_list()[0], sky_df['bad_ind'].to_list()[0], badpx = sky_df['badpx_ind'].to_list()[0], 
               sky = sky_df['calc_sky'].item(), rms = sky_df['calc_rms'].item(), show = False, 
               save = True, savepath = '../inspection_plots/{r}_prof_insplot.pdf'.format(r = root), 
               title = root)

0 jw02883007001_02101_00001_nrca1
['NRCA1'] ['F182M']
1 jw02883007001_02101_00001_nrca2
['NRCA2'] ['F182M']
2 jw02883007001_02101_00001_nrca3
['NRCA3'] ['F182M']
3 jw02883007001_02101_00001_nrca4
['NRCA4'] ['F182M']
4 jw02883007001_02101_00001_nrcb1
['NRCB1'] ['F182M']
5 jw02883007001_02101_00001_nrcb2
['NRCB2'] ['F182M']
6 jw02883007001_02101_00001_nrcb3
['NRCB3'] ['F182M']
7 jw02883007001_02101_00001_nrcb4
['NRCB4'] ['F182M']
8 jw02883007001_02101_00002_nrca1
['NRCA1'] ['F182M']
9 jw02883007001_02101_00002_nrca2
['NRCA2'] ['F182M']
10 jw02883007001_02101_00002_nrca3
['NRCA3'] ['F182M']
11 jw02883007001_02101_00002_nrca4
['NRCA4'] ['F182M']
12 jw02883007001_02101_00002_nrcb1
['NRCB1'] ['F182M']
13 jw02883007001_02101_00002_nrcb2
['NRCB2'] ['F182M']
14 jw02883007001_02101_00002_nrcb3
['NRCB3'] ['F182M']
15 jw02883007001_02101_00002_nrcb4
['NRCB4'] ['F182M']
16 jw02883007001_02101_00003_nrca1
['NRCA1'] ['F182M']
17 jw02883007001_02101_00003_nrca2
['NRCA2'] ['F182M']
18 jw02883007001_021

/Users/rortiziii/Work/darksky/darksky/measureskyregion.py:33: RuntimeWarning: Mean of empty slice
  loci=np.nanmean(newdat)
/Users/rortiziii/anaconda3/envs/skysb/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/rortiziii/Work/darksky/darksky/measureskyregion.py:40: RuntimeWarning: All-NaN slice encountered
  loci=np.nanmedian(newdat)
/Users/rortiziii/anaconda3/envs/skysb/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1384: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
/Users/rortiziii/Work/darksky/darksky/measureskyregion.py:43: RuntimeWarning: All-NaN slice encountered
  return np.nanmedian(newdat),sigi


32 jw02883007001_03101_00001_nrca1
['NRCA1'] ['F182M']
33 jw02883007001_03101_00001_nrca2
['NRCA2'] ['F182M']


/Users/rortiziii/Work/darksky/darksky/measureskyregion.py:33: RuntimeWarning: Mean of empty slice
  loci=np.nanmean(newdat)
/Users/rortiziii/anaconda3/envs/skysb/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/rortiziii/Work/darksky/darksky/measureskyregion.py:40: RuntimeWarning: All-NaN slice encountered
  loci=np.nanmedian(newdat)
/Users/rortiziii/anaconda3/envs/skysb/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1384: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
/Users/rortiziii/Work/darksky/darksky/measureskyregion.py:43: RuntimeWarning: All-NaN slice encountered
  return np.nanmedian(newdat),sigi


34 jw02883007001_03101_00001_nrca3
['NRCA3'] ['F182M']
35 jw02883007001_03101_00001_nrca4
['NRCA4'] ['F182M']
36 jw02883007001_03101_00001_nrcalong
['NRCALONG'] ['F360M']
37 jw02883007001_03101_00001_nrcb1
['NRCB1'] ['F182M']
38 jw02883007001_03101_00001_nrcb2
['NRCB2'] ['F182M']
39 jw02883007001_03101_00001_nrcb3
['NRCB3'] ['F182M']
40 jw02883007001_03101_00001_nrcb4
['NRCB4'] ['F182M']
41 jw02883007001_03101_00001_nrcblong
['NRCBLONG'] ['F360M']
42 jw02883007002_02101_00001_nrca1
['NRCA1'] ['F182M']
43 jw02883007002_02101_00001_nrca2
['NRCA2'] ['F182M']
44 jw02883007002_02101_00001_nrca3
['NRCA3'] ['F182M']
45 jw02883007002_02101_00001_nrca4
['NRCA4'] ['F182M']
46 jw02883007002_02101_00001_nrcalong
['NRCALONG'] ['F360M']
47 jw02883007002_02101_00001_nrcb1
['NRCB1'] ['F182M']
48 jw02883007002_02101_00001_nrcb2
['NRCB2'] ['F182M']
49 jw02883007002_02101_00001_nrcb3
['NRCB3'] ['F182M']
50 jw02883007002_02101_00001_nrcb4
['NRCB4'] ['F182M']
51 jw02883007002_02101_00001_nrcblong
['NRCBLON

In [6]:
fits.open(file)[3].header

XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                   32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2048 / length of data axis 1                          
NAXIS2  =                 2048 / length of data axis 2                          
PCOUNT  =                    0 / required keyword; must = 0                     
GCOUNT  =                    1 / required keyword; must = 1                     
EXTNAME = 'PIXELMASK'          / extension name                                 